### Vinho Verde 🍾

In the folder `/ databricks-datasets / wine-quality` there's the ` winequality-white.csv` file. The dataset describes variants of the Portuguese white wine _Vinho Verde_.

#### Part 1
  - count the number of wine variants for each `quality` level (value between` 1` and `8`). Then sort the count by ascending `quality`. Example

| quality | count |
|--------|------|
| 1 | 123 |
| 2 | 456 |
| ... | ... |

  - save the result in parquet format in the following path `/ exam / 221017_wine_part_1.parquet`

#### Part 2
- consider only the wine variants whose residual sugar is less than 9
- for each level of `quality`, calculate
  - the maximum `pH` in the` pHMax` column
  - the average of the `density` in the` densityAvg` column
- then sort by increasing `quality` level

Example

| quality |  pHMax | densityAvg |
|--------|------|------|
| 1 | 3.5 | 0.99 |
| 2 | 3.6 | 0.98 |
| ... | ... |... |

- save the result in parquet format in the following path `/ exam / 221017_wine_part_2.parquet`

### IoT Devices 📶

The `databricks-datasets / iot / iot_devices.json` file collects data detected by IoT devices on March 20, 2016. The` timestamp` column contains the [Unix time](https://en.wikipedia.org/wiki/Unix_time ) expressed however in **milliseconds**.

- From the `timestamp` column, extract the hour and minute of the day.
- Then calculate the record count for each hour and minute pair.
- Sort by increasing hour and minute.

Example

| hour |  minute | count |
|--------|------|------|
| 0 | 4 | 99 |
| 1 | 2 | 8 |
| ... | ... |... |

- save the result in parquet format in the following path `/exam/221017_iot.parquet`

### PART 1

In [0]:
df = spark.read.csv("dbfs:/databricks-datasets/wine-quality/winequality-white.csv",header=True, sep=";")

In [0]:
result=( 
    df.groupBy(df.quality)
    .count()
    .sort(df.quality)
)

In [0]:
%fs mkdirs exam

res0: Boolean = true

In [0]:
%fs ls

path,name,size,modificationTime
dbfs:/checkpoints/,checkpoints/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/exam/,exam/,0,0
dbfs:/local_disk0/,local_disk0/,0,0
dbfs:/output/,output/,0,0


In [0]:
dbutils.fs.rm("/exam/221017_wine_part_1.parquet", recurse=True)

Out[3]: True

In [0]:
result.write.format("parquet").save("/exam/221017_wine_part_1.parquet")

In [0]:
display(spark.read.parquet("/exam/221017_wine_part_1.parquet"))

quality,count
3,20
4,163
5,1457
6,2198
7,880
8,175
9,5


### PART 2

In [0]:
df = spark.read.csv("dbfs:/databricks-datasets/wine-quality/winequality-white.csv",header=True, sep=";")

In [0]:
from pyspark.sql.functions import col

result = ( 
    df.where(col("residual sugar")<9)
    .groupBy(df.quality)
    .agg(
        {"pH": "max",
         "density":"mean"})
    .sort(df.quality)
    .withColumnRenamed("max(pH)", "pHMax")
    .withColumnRenamed("avg(density)", "densityAvg")
)

In [0]:
dbutils.fs.rm("/exam/221017_wine_part_2.parquet", recurse=True)

Out[8]: True

In [0]:
result.write.format("parquet").save("/exam/221017_wine_part_2.parquet")

In [0]:
display(spark.read.parquet("/exam/221017_wine_part_2.parquet"))

quality,densityAvg,pHMax
3,0.9935253846153846,3.55
4,0.9935494029850745,3.72
5,0.9938479662802943,3.79
6,0.9926420051085587,3.81
7,0.9914311690140855,3.82
8,0.9911207299270076,3.59
9,0.9900749999999999,3.41


### PART 3

In [0]:
df = spark.read.json("dbfs:/databricks-datasets/iot/iot_devices.json")

In [0]:
from pyspark.sql.functions import from_unixtime, hour, minute

result = (
    df.select(df.timestamp)
    .withColumn('date', from_unixtime((col('timestamp')/1000)))
    .withColumn("hour", hour(col("date")))
    .withColumn("minute", minute(col("date")))
    .groupBy(col("hour"),col("minute"))
    .count()
   .sort(col("hour"),col("minute"))
)

In [0]:
dbutils.fs.rm("/exam/221017_iot.parquet", recurse=True)

Out[13]: True

In [0]:
result.write.format("parquet").save("/exam/221017_iot.parquet")

In [0]:
display(spark.read.parquet("/exam/221017_iot.parquet"))

hour,minute,count
3,20,155936
3,21,42228
